# 05 - Proyecto Individual: Sistema de recomendación.

## Carga de las librerías requeridas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Carga del dataset `ml_games`.

In [2]:
%cd /content/drive/MyDrive/AI Projects/PI01

/content/drive/MyDrive/AI Projects/PI01


In [3]:
ml_games = pd.read_parquet('datasets/ml_games.parquet').reset_index(drop=True)
ml_games.head(n=10)

id                  item_name  \
0   10             Counter-Strike   
1   20      Team Fortress Classic   
2   30              Day of Defeat   
3   40         Deathmatch Classic   
4   50  Half-Life: Opposing Force   
5   60                   Ricochet   
6   70                  Half-Life   
7  130      Half-Life: Blue Shift   
8  300      Day of Defeat: Source   
9  240     Counter-Strike: Source   

                                        bag_of_words  
0  action action fps multiplayer shooter classic ...  
1  action action fps multiplayer classic shooter ...  
2  action fps world war ii multiplayer action sho...  
3  action action fps multiplayer classic shooter ...  
4  action fps action sci-fi singleplayer classic ...  
5  action action fps multiplayer first-person cyb...  
6  action fps classic action sci-fi singleplayer ...  
7  action fps action sci-fi singleplayer shooter ...  
8  action fps world war ii multiplayer action sho...  
9  action action fps multiplayer shooter team-bas...

In [4]:
ml_games.shape

(8560, 3)

Reduciremos el dataset para poder almacenar la matriz de similitud que se creará más adelante.

In [42]:
np.random.seed(42)
ml_games = ml_games.sample(frac=.90)

In [43]:
ml_games.shape

(7704, 3)

## Vectorización del campo `bag_of_words`.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
vectorizer = TfidfVectorizer()
vectorizer.fit(ml_games.bag_of_words)

TfidfVectorizer()

In [45]:
vec_words = vectorizer.transform(ml_games.bag_of_words)

In [46]:
vec_words.shape

(7704, 413)

### Cálculo de la matriz de similaridad.

Se usará el *coseno de similaridad*.

In [47]:
from sklearn.metrics.pairwise import linear_kernel

In [48]:
similarity_matrix = linear_kernel(vec_words, vec_words)

In [49]:
similarity_matrix.shape

(7704, 7704)

In [50]:
similarity_matrix = similarity_matrix.astype('float16')

### Desarrollo de la función de recomendación.

In [51]:
def gameRecommendation(id):
  similarity_matrix_idx = ml_games.query('id == @id').index[0]
  similarity_matrix_row = similarity_matrix[similarity_matrix_idx]
  game_row_idxs = similarity_matrix_row.argsort()[-6:-1][::-1]

  ref_game = ml_games.query('id == @id')[['id', 'item_name']].to_dict(orient='records')[0]
  top_5 = ml_games[['id', 'item_name']].iloc[game_row_idxs].to_dict(orient='records')
  return {'game':ref_game, 'top_5': top_5}

In [53]:
gameRecommendation(130)

{'game': {'id': 130, 'item_name': 'Half-Life: Blue Shift'},
 'top_5': [{'id': 365330, 'item_name': 'Jump/Boxer'},
  {'id': 401280, 'item_name': "Nubs' Adventure"},
  {'id': 367050, 'item_name': 'Vindicator: Uprising'},
  {'id': 454100, 'item_name': 'Dungeon Escape'},
  {'id': 315940, 'item_name': 'Top Hat'}]}

## Guardado de la matriz de similitud y el dataset `ml_games`.

In [54]:
np.savez_compressed('datasets/similarity_matrix.npz', similarity_matrix)

In [55]:
!ls -sh datasets/similarity_matrix.npz

97M datasets/similarity_matrix.npz


In [56]:
ml_games.to_parquet('datasets/ml_games.parquet', index=True)